# Tune Experiment Tracking w/ MLflow

In [1]:
import os
import tempfile
import time

import mlflow

from ray import train, tune
from ray.air.integrations.mlflow import MLflowLoggerCallback, setup_mlflow

In [2]:
def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100)**(-1) + height * 0.1

def train_function(config):
    width, height = config['width'], config['height']

    for step in range(config.get('steps', 100)):
        intermediate_score = evaluation_fn(step, width, height)
        train.report({'iterations':step, 'mean_loss': intermediate_score})
        time.sleep(0.1)

In [ ]:
mlflow_uri='http://127.0.0.1:5000'

def tune_with_callback(mlflow_tracking_uri, finish_fast=False):
    tuner = tune.Tuner(
        train_function,
        tune_config=tune.TuneConfig(num_samples=5),
        run_config=train.RunConfig(
            name='mlflow',
            callbacks=[
                MLflowLoggerCallback(
                    tracking_uri=mlflow_tracking_uri,
                    experiment_name='mlflow_tune_callback_example',
                    save_artifact=True,
                )]
            ),
        param_space={
            'width':tune.randint(10, 100),
            'height': tune.randint(0, 100),
            'steps': 5 if finish_fast else 10
        }
    )

    results = tuner.fit()

#tune_with_callback(mlflow_tracking_uri=mlflow_uri)

2024/11/12 19:50:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run train_function_3a097_00004 at: http://127.0.0.1:5000/#/experiments/759485979032750589/runs/0c69659c3ed74b42b66009882dc27399.
2024/11/12 19:50:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/759485979032750589.
2024/11/12 19:50:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run train_function_3a097_00002 at: http://127.0.0.1:5000/#/experiments/759485979032750589/runs/1473399c11204ea1b54fc38fcb5253eb.
2024/11/12 19:50:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/759485979032750589.
2024/11/12 19:50:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run train_function_3a097_00001 at: http://127.0.0.1:5000/#/experiments/759485979032750589/runs/6b1b4864db30400b94d4c9921eaf85e0.
2024/11/12 19:50:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0

In [ ]:
mlflow_uri='http://127.0.0.1:5000'
# Using setup_mlflow
def train_function_mlflow(config):
    tracking_uri = config.pop("tracking_uri", None)
    setup_mlflow(
        config,
        run_name='test_run_2'
    )

    mlflow.set_tracking_uri(mlflow_uri) # Works if instantiated outside
    mlflow.set_experiment('setup_mlflow_example') # Works if instantiated outside
    mlflow.log_params(config)

    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config.get("steps", 100)):
        intermediate_score = evaluation_fn(step, width, height)
        # Log the metrics to mlflow
        mlflow.log_metrics(dict(mean_loss=intermediate_score), step=step)
        train.report({"iterations": step, "mean_loss": intermediate_score})
        time.sleep(0.1)


def tune_with_setup(mlflow_tracking_uri, finish_fast=False):
    # Set the experiment, or create a new one if does not exist yet.
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    mlflow.set_experiment(experiment_name="setup_mlflow_example")

    tuner = tune.Tuner(
        train_function_mlflow,
        tune_config=tune.TuneConfig(num_samples=5),
        run_config=train.RunConfig(
            name="mlflow",
        ),
        param_space={
            "width": tune.randint(10, 100),
            "height": tune.randint(0, 100),
            "steps": 5 if finish_fast else 100,
            "tracking_uri": mlflow_uri,
        },
    )
    results = tuner.fit()

#tune_with_setup(mlflow_uri)

2024-11-12 20:57:30,130	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/JCA/ray_results/mlflow' in 0.0399s.
2024-11-12 20:57:30,192	INFO tune.py:1041 -- Total run time: 22.66 seconds (22.52 seconds for the tuning loop).
